In [407]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
import k3d



In [408]:
mag = lambda v : np.sqrt(np.sum([i*i for i in v]))

In [410]:
def norm(v):
    m = mag(v)
    vN = [float(i)/float(m) for i in v]
    return vN

In [385]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [386]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [387]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [388]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [389]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [390]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

In [391]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
            
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [392]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [393]:
def getArcAngle(dist,rad):
    return dist/rad

In [426]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        if np.sum([j*j for j in v3])== 0.0:
            #it's parallel!
            particle = addLigand(particle,eps,v,lock)
            continue
        dp = np.dot(v1,v2)
        #the magic sauce :)
        ang = np.arctan2(mag(np.cross(v1,v2)), np.dot(v1,v2))
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [427]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing)
    return particle

In [438]:
p = NanoParticle()
m = 3
step = np.pi*2/float(m)
for i in range(m):
    p = addPatch(p,6,4,(i*step,np.pi/2.0),5,1.6,lock=True)
p = addPatch(p,6,4,(np.pi/2.0,0),5,1.6,lock=True)
p = addPatch(p,6,4,(3*np.pi/2.0,0),5,1.6,lock=True)
p = addBelt(p,6,4,10,0,lock=True)

In [439]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[120968101008, 120967862800, 120968100624, 120968098384, 120968099920, 120968098896, 120968100176, 120967426128, 120968100368, 120967862480, 120968101712, 120967427728, 120968100112, 120968277776, 120968099536, 120968277392, 120968275024, 120968100240, 120968275408, 120967860304, 120968274576, 120968275920, 120968274960, 120968275216, 120968275280, 120968274704, 120968276880, 120968276368, 120968276944, 120968277264, 120968276752, 120968277072, 120968276240, 120968276304, 120968277328, 120968275088, 120968271696, 120968271888, 120968271312, 120968272336, 120968272656, 120968271184, 120968276496, 120968272144, 120968273296, 120968271376, 120968273616, 120968272208, 120968277200, 120968271120, 120968273808, 120968273424, 120968272976, 120968273680, 120968273552, 120968573072, 120968272848, 120968574288, 120968573648, 120968272080, 120968273040, 120968574352, 120968573840, 120968573584, 120968575056, 120968272400, 120968574672, 120968574992, 120968573328, 120968575120, 120968100304])